In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta, timezone
import requests
import os
from time import sleep
import pytz
from scipy.stats import pearsonr
from Functions import check_correlation, analyze_events, wrap_analyze_events, load_data, data_preparation, wrap_analyze_events_multiprocessing, show_output, start_wrap_analyze_events_multiprocessing, get_volume_info
import pandas as pd
pd.set_option('display.max_rows', None)
from time import time
from multiprocessing import Process
from multiprocessing import Lock, Pool, Manager
from time import time

In [2]:
start_interval=datetime(2023,5,11)
end_interval=datetime.now()

filter_position=(0,300)

data, df, df_obj = load_data(start_interval=start_interval, end_interval=end_interval, filter_position=filter_position)
df

StatusCode for getting most_traded_coins: 200
new Json saved in /home/alberto/Docker/Trading/tracker/json/most_traded_coins.json
StatusCode for getting get-volumeinfo: 200
['/home/alberto/Docker/Trading/analysis/json/data-15-5-2023-11743.json']
Retrieving data from /home/alberto/Docker/Trading/analysis/json/data-15-5-2023-11743.json
122 coins have been loaded


,n_observations,position,vol_Mill_1w
BTCUSDT,7143.0,1.0,1104.174188
ETHUSDT,7143.0,2.0,707.430271
XRPUSDT,7143.0,3.0,112.420556
ARBUSDT,7143.0,4.0,106.272546
PEPEUSDT,7143.0,5.0,103.363080
LTCUSDT,7143.0,6.0,96.194553
SUIUSDT,7142.0,7.0,93.163614
EDUUSDT,7142.0,9.0,76.008748
BNBUSDT,7143.0,11.0,63.763113
SOLUSDT,7143.0,12.0,60.521163


In [ ]:
volume_info = get_volume_info()
volume_info['most_traded_coins']

In [ ]:
# WRAP ANALYSIS EVENTS
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m']
list_minutes = [5, 15, 30, 60, 120, 180, 360]
list_event_buy_volume = [0.7, 0.8]
list_event_volume = [5,6]

resp = wrap_analyze_events(data, list_buy_vol, list_vol, list_minutes, list_event_buy_volume,
                    list_event_volume, start_interval, end_interval)


In [ ]:
# WRAP ANALYSIS EVENTS MULTIPROCESSING
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m']
list_minutes = [5, 15, 30, 60, 120, 180, 360]
list_event_buy_volume = [0.65, 0.7, 0.8]
list_event_volume = [3,4,5,6]
n_processes = 6

shared_data = start_wrap_analyze_events_multiprocessing(data, list_buy_vol, list_vol, 
                                                        list_minutes, list_event_buy_volume,
                                                        list_event_volume, start_interval,
                                                        end_interval, n_processes)


In [ ]:
# SHOW DATA FROM MULTIPROCESSING
output = show_output(shared_data)
df = pd.DataFrame(output).transpose()
df.sort_values(by=['lower_bound'], ascending=False)

In [ ]:
df = pd.DataFrame(resp).transpose()
lb = []
for index, row in df.iterrows():
    mean = row['mean']
    std = row['std']
    lb.append(mean - std)
    
df['lb'] = lb
    
df.sort_values(by=['lb'], ascending=False)

In [ ]:
output = show_output(shared_data)
df = pd.DataFrame(output).transpose()
df